<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/Grad_CAMTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for Grad-CAM
Note: Need to add our own trained model first for it to work.

what it does?


1.   Visiulases how the CNN is making certain decisions
2.   It will highlight which areas of our image it is taking into account to make its decision.



In [4]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load your trained autonomous car model
model = tf.keras.models.load_model("mobnet.h5")

# Function to preprocess input image
def process_image(image_path, resized_shape=(224, 224)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, resized_shape)
    image = image / 255.0  # Normalise pixel values to [0,1]
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Grad-CAM function
def compute_gradcam(model, img_array, layer_name="conv2d"):
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions  # Assuming regression (steering angle)

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = np.maximum(heatmap[0], 0)  # ReLU activation
    heatmap /= np.max(heatmap)  # Normalize between 0 and 1
    return heatmap

# Overlay Grad-CAM heatmap on original image
def overlay_heatmap(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)  # Convert to 0-255 scale
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply color map
    overlayed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)

    return overlayed_img

# Load an example image from your dataset
image_path = "/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/1.png"  # Change to an actual image path
img_array = process_image(image_path)

# Compute Grad-CAM heatmap
heatmap = compute_gradcam(model, img_array, layer_name="classification")  # Change "conv2d" to your CNN layer

# Overlay the heatmap on the original image
overlayed_image = overlay_heatmap(image_path, heatmap)

# Display the results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.imread(image_path)[:, :, ::-1])
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(overlayed_image)
plt.title("Grad-CAM Overlay")

plt.show()


2025-05-07 12:43:57.279633: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at reduction_ops_common.h:147 : INVALID_ARGUMENT: Invalid reduction dimension (2 for input with 2 dimension(s)


InvalidArgumentError: {{function_node __wrapped__Mean_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid reduction dimension (2 for input with 2 dimension(s) [Op:Mean]

In [5]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load your trained autonomous car model
model = tf.keras.models.load_model("mobnet.h5")

# Function to preprocess input image
def process_image(image_path, resized_shape=(224, 224)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, resized_shape)
    image = image / 255.0  # Normalise pixel values to [0,1]
    # Add batch dimension
    image = tf.expand_dims(image, axis=0)
    return image

# Grad-CAM function
def compute_gradcam(model, img_array, layer_name):
    # First, ensure the specified layer exists
    try:
        grad_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=[model.get_layer(layer_name).output, model.output]
        )
    except ValueError:
        # If layer not found, find a suitable convolutional layer
        conv_layers = [layer.name for layer in model.layers if 'conv' in layer.name.lower()]
        if not conv_layers:
            raise ValueError("No convolutional layer found in the model")
        print(f"Layer '{layer_name}' not found. Using '{conv_layers[-1]}' instead.")
        layer_name = conv_layers[-1]
        grad_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=[model.get_layer(layer_name).output, model.output]
        )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if isinstance(predictions, list):
            predictions = predictions[0]  # Handle multiple outputs
        
        # For regression task (steering angle)
        target_output = predictions  # Use the raw prediction

    # Calculate gradients
    grads = tape.gradient(target_output, conv_outputs)
    
    # Global average pooling
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight the channels by importance
    weighted_conv_outputs = tf.multiply(conv_outputs[0], pooled_grads)
    
    # Average across feature maps
    heatmap = tf.reduce_mean(weighted_conv_outputs, axis=-1)
    
    # ReLU activation & normalization
    heatmap = np.maximum(heatmap, 0)
    if np.max(heatmap) > 0:  # Avoid division by zero
        heatmap /= np.max(heatmap)
    
    return heatmap

# Overlay Grad-CAM heatmap on original image
def overlay_heatmap(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Could not read image at {img_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize heatmap to match the original image dimensions
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert heatmap to RGB colormap
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # Convert ColorMap from BGR to RGB
    
    # Overlay the heatmap on original image
    overlayed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    
    return overlayed_img

# Display function
def display_gradcam(image_path, model, target_layer=None):
    # Process the image
    img_array = process_image(image_path)
    
    # If no target layer specified, try to find a suitable one
    if target_layer is None:
        # Find the last convolutional layer
        conv_layers = [layer.name for layer in model.layers if 'conv' in layer.name.lower()]
        if conv_layers:
            target_layer = conv_layers[-1]
            print(f"Using layer: {target_layer}")
        else:
            # Try to find any layer that might work
            suitable_layers = [layer.name for layer in model.layers 
                             if len(layer.output_shape) == 4]  # 4D output should be a feature map
            if suitable_layers:
                target_layer = suitable_layers[-1]
                print(f"Using layer: {target_layer}")
            else:
                raise ValueError("Couldn't find a suitable layer for Grad-CAM")
    
    # Compute Grad-CAM
    heatmap = compute_gradcam(model, img_array, layer_name='classification')
    
    # Get model prediction
    prediction = model.predict(img_array)
    if isinstance(prediction, list):
        prediction = prediction[0]  # Handle multiple outputs
    
    # Create the visualization
    overlayed_image = overlay_heatmap(image_path, heatmap)
    
    # Display results
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 3, 1)
    original_img = cv2.imread(image_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(heatmap, cmap='jet')
    plt.title("Grad-CAM Heatmap")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(overlayed_image)
    plt.title(f"Overlay (Prediction: {prediction[0]:.4f})")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return heatmap, overlayed_image, prediction

# Example usage
if __name__ == "__main__":
    image_path = "/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/1.png"
    
    # Print model summary to find layers
    print("Model layers:")
    model.summary()
    
    # Try to automatically find a good layer or specify a known one
    # For MobileNet, a good choice might be the last convolutional layer
    last_conv_layer = None
    for layer in model.layers:
        if 'conv' in layer.name.lower():
            last_conv_layer = layer.name
    
    print(f"\nSelected layer: {last_conv_layer}")
    
    # Generate and display Grad-CAM
    heatmap, overlay, prediction = display_gradcam(
        image_path=image_path,
        model=model,
        target_layer=last_conv_layer  # Specify layer or leave as None for auto-detection
    )
    
    print(f"Model prediction: {prediction}")

Model layers:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 MobilenetV3small (Function  (None, 7, 7, 576)            939120    ['input[0][0]']               
 al)                                                                                              
                                                                                                  
 global_avg_pool (GlobalAve  (None, 576)                  0         ['MobilenetV3small[0][0]']    
 ragePooling2D)                                                                                   
                                                                                

2025-05-07 12:56:23.165718: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at reduction_ops_common.h:147 : INVALID_ARGUMENT: Invalid reduction dimension (2 for input with 2 dimension(s)


InvalidArgumentError: {{function_node __wrapped__Mean_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid reduction dimension (2 for input with 2 dimension(s) [Op:Mean]

In [6]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load model
model = tf.keras.models.load_model("mobnet.h5")

# Print model summary to find usable layers
print("Model structure:")
model.summary()

def create_gradcam(img_path, model, last_conv_layer_name, pred_index=None):
    # Preprocess the image
    img_array = preprocess_image(img_path)
    
    # Create a model that maps the input image to:
    # 1. The activations of the last conv layer
    # 2. The model's predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    # Then, we compute the gradient of the predicted output with respect to the feature map
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        
        # We use the first output (regression value) for gradient calculation
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        
        # This is the regression value
        target_output = preds[:, pred_index]
    
    # The gradient of the output with respect to the last conv layer output
    grads = tape.gradient(target_output, last_conv_layer_output)
    
    # Vector of shape (channels,) - importance weights for each channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Shape: (height, width, channels)
    last_conv_layer_output = last_conv_layer_output[0]
    
    # Multiply each channel by how important it is
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    
    # Remove the last dimension and normalize
    heatmap = tf.squeeze(heatmap)
    
    # Normalize to [0, 1]
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    # Convert to numpy and return
    return heatmap.numpy(), preds.numpy()

def preprocess_image(img_path, target_size=(224, 224)):
    # Read and resize the image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    # Expand dimensions for batch and normalize
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    return img_array

def save_and_display_gradcam(img_path, heatmap, prediction, alpha=0.4):
    # Load the original image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # Superimpose the heatmap on original image
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    
    # Display the original image, heatmap, and combined image
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("Original Image")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(heatmap)
    plt.title("Grad-CAM Heatmap")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(superimposed_img)
    pred_val = prediction[0][0] if isinstance(prediction[0], np.ndarray) else prediction[0]
    plt.title(f"Overlay (Pred: {pred_val:.4f})")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return superimposed_img

# Try to identify reasonable layers to use
def find_conv_layers(model):
    conv_layers = []
    for i, layer in enumerate(model.layers):
        # Look for conv layers
        if 'conv' in layer.name.lower() and hasattr(layer, 'output_shape'):
            if len(layer.output_shape) == 4:  # Should be 4D for feature maps
                conv_layers.append((i, layer.name, layer.output_shape))
    
    return conv_layers

# Find convolutional layers
conv_layers = find_conv_layers(model)
print("\nPotential layers for Grad-CAM:")
for idx, name, shape in conv_layers:
    print(f"{idx}: {name} (shape: {shape})")

# Select a layer to use (usually one of the last conv layers)
# Choose based on the model summary - this is just a placeholder
try:
    # Try to find a good layer automatically
    # MobileNet typically has layers with names like 'conv_pw_13' near the end
    last_conv_names = ['conv_pw_13', 'block_16_project', 'Conv_1']
    
    # Find the first matching layer
    last_conv_layer = None
    for name_pattern in last_conv_names:
        matching_layers = [l[1] for l in conv_layers if name_pattern in l[1]]
        if matching_layers:
            last_conv_layer = matching_layers[0]
            break
    
    # If no match found, use the last conv layer
    if not last_conv_layer and conv_layers:
        last_conv_layer = conv_layers[-1][1]
    
    if last_conv_layer:
        print(f"\nUsing layer: {last_conv_layer}")
        
        # Image path (replace with your actual path)
        img_path = "/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/1.png"
        
        # Generate Grad-CAM
        heatmap, prediction = create_gradcam(
            img_path=img_path,
            model=model,
            last_conv_layer_name=last_conv_layer
        )
        
        # Display the results
        save_and_display_gradcam(img_path, heatmap, prediction)
        
        print(f"Model prediction: {prediction}")
    else:
        print("No suitable convolutional layer found for Grad-CAM")

except Exception as e:
    print(f"Error generating Grad-CAM: {str(e)}")
    import traceback
    traceback.print_exc()

Model structure:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 MobilenetV3small (Function  (None, 7, 7, 576)            939120    ['input[0][0]']               
 al)                                                                                              
                                                                                                  
 global_avg_pool (GlobalAve  (None, 576)                  0         ['MobilenetV3small[0][0]']    
 ragePooling2D)                                                                                   
                                                                             